# Kwirk

https://en.wikipedia.org/wiki/Kwirk

Kwirk o Puzzle Boy como se le conoce en Japón es un juego de puzzle publicado en 1989. 


Es el típico juego de niveles en el que los personajes (1 o 2 según el nivel) tienen que ir de un punto A (origen) a un punto B (meta) y para ello tiene que mover cajas, empujarlas al agua o abrir/cerrar puertas. Cuando hay 2 jugadores, estos se pueden intercambiar para poder llegar ambos a la meta sin que queden atrapados.


En el ejemplo de abajo los dos personajes tendrían que salir por las escaleras.




In [1]:
from IPython.display import Image
Image(url='./kwirkEjemplo.jpg')  

La versión que vamos a crear va a tener una serie de simplificaciones:

- Todas las cajas van a ser de tamaño 1x1. En el juego original las cajas podian tener un tamaño arbitrario (1x2, 2x2, 5x1 ...)
- No hay puertas que abrir/cerrar.
- Puede haber dos jugadores.
- Puede haber agua, el agua se tapa con las cajas.

En el ejemplo de arriba tenemos un nivel con:
- unas paredes (los ladrillos)
- una caja que se puede mover (de color más claro), de tamaño 1x1 (así van a ser todos en nuestra versión)
- dos jugadores
- una casilla de agua (en negro, muy cerca del jugador que tiene una gorra.)
- La meta (en forma de escalera)

-----------------

# Estructuras de datos

En todos los juegos de tipo puzzle (kwirk, sokoban, bomberman, bloxorz ...) siempre tenemos una parte del nivel que es inmutable (casillas libres, paredes, destinos o metas) y una parte que cambia, a la que llamamos estado (jugador, cajas que se puedan mover, agua que se pueda tapar etc).

En un problema de búsqueda en espacio de estados se opera con dicho estado para encontrar la secuencia de pasos que nos lleva del estado inicial al estado final.


En un nivel de Kwirk tenemos los siguientes elementos inmutables:
1. Tablero: Un mapa de 2 dimensiones con casillas libres o paredes. 
1. Meta o destino: La coordenada a la que tienen que llegar los jugadores.

Y tenemos los siguientes elementos mutables o que cambian, (el estado)
1. Posición jugadores.
2. Turno del jugador que se está moviendo.
3. Cajas que se pueden mover.
4. Agua. Casillas de agua que podemos tapar con cajas



Las estructuras de datos que se van a usar son:
- Tablero: Tupla de 2-dimensiones con 0 (casilla libre) ó 1 (paredes). No cambia -> tupla
- Meta: Una tupla con una coordenada.
- Cajas: Un set con multiples coordenadas (y,x) que serían tuplas. Cambia y no importa el orden -> set
- Personaje: Una lista [y,x]. Como tenemos 2 pues una lista de listas [[y1,x1],[y2,x2]]. Cuando un jugador no existe en un determinado nivel o ya ha entrado en la meta se sustituye por None.
- El turno un booleano. True se mueve el jugador 1, False se mueve el jugador 2.
- Agua: Un set con multiples coordenadas (y,x) que serían tuplas. Cambia y no importa el orden -> set

### Define a mano los datos del siguiente ejemplo

In [2]:
Image(url='./nivel_ejemplo.png')  

El dibujo siguiente es una captura generada con el juego que vamos a hacer. Aparece el jugador 1 (Gazpacho) y el jugador 2 (Pincho). La meta es la estrella, vemos que para poder resolver el mapa el jugador 2 tendría que empujar la caja hasta el agua, para taparla y que el jugador 1 pudiera pasar.

Es un tablero de 5 filas x 6 columnas.

Vamos a codificar todas las coordenadas como [y,x].
Las filas empiezan a contar arriba y las columnas empiezan a contar a la izquierda.

#### Ejercicio 1

Termina de representar el nivel anterior.

In [4]:
board = ((1,1,1,1,1,1),
         (1,0,0,1,0,1),
         (1,0,0,0,0,1),
         (1,0,1,0,0,1),
         (1,1,1,1,1,1))
destination = (1,4)

players = [[3,4],[2,1]]
boxes = set([(2,2)])
water = set([(2,4)])
turn = True

## Funciones para recuperar el contenido de los elementos del juego.

Necesitamos una función para conocer el contenido del juego en una coordenada determinada. Va a servir para dos cosas:
- Simplificar las funciones que determinen los movimientos válidos.
- Simplificar el proceso de dibujado del tablero (y hacerlo válido para más juegos, luego veremos como).

En una determinada posición puede haber: 
- Un elemento del nivel que no cambia (hueco, pared, destino)
- Un elemento del estado que es cambiante (jugadores, cajas, agua)


Se van a usar los siguientes elementos:
"grass","wall","destination","box","water", "player1", "player2" y None

En una casilla podemos tener ["grass","box"] o ["grass",None] o ["wall",None] etc. Queremos una función que devuelva ese tipo de combinaciones.

Significado:
- grass: hierba, hueco por el que se puede caminar.
- wall: pared, parte del tablero por la que no se puede caminar.
- destination: destino, coordenada del tablero a donde hay que llegar
- player1 y player2 son las coordenadas de los jugadores
- box: Caja, elemento del juego que se puede empujar
- water: Agua, elemento del juego que podemos tapar con una caja.

#### Ejercicio 2

Completa la siguiente función

In [12]:
def get_content(coord):
    """
    Obtiene el contenido de una determinada posición. Consulta para ello los valores de las variables del ejercicio1
    
    Parameters
    ----------
    coord : Posición [y,x] de la que queremos conocer el contenido
    
    Returns
    --------
    contenido : Una lista de tamaño 2 con el elemento del nivel y del estado en la coordenada
    """
    contenido = [None,None]
    
    if coord == destination:
        contenido[0] = "destination"
    elif board[coord[0]][coord[1]] == 0:
        contenido[0] = "grass"
    else:
        contenido[0] = "wall"
    
    if coord[0] == players[0][0] and coord[1] == players[0][1]:
        contenido[1] = "player1"
    elif coord[0] == players[1][0] and coord[1] == players[1][1]:
        contenido[1] = "player2"
    elif coord in water:
        contenido[1] = "water"
    elif coord in boxes:
        contenido[1] = "box"
    
    return contenido


'''
Debe devolver
['grass', 'player2']
['grass', 'player1']
['grass', 'water']
['grass', 'box']
['wall', None]
['destination', None]
'''
print(get_content((2,1)))
print(get_content((3,4)))
print(get_content((2,4)))
print(get_content((2,2)))
print(get_content((4,1)))
print(get_content((1,4)))






['grass', 'player2']
['grass', 'player1']
['grass', 'water']
['grass', 'box']
['wall', None]
['destination', None]


## Funciones básicas de comprobación de casillas

Antes de crear funciones para mover el jugador o empuzar las cajas se necesitan funciones básicas que:
- compruebe si una casilla es valida: está dentro de los límites del tablero y hay un hueco.
- compruebe si una casilla esta libre para un jugador: valida, no hay cajas, no hay agua, no está el otro jugador.En esas casillas se puede mover el jugador.
- compruebe si una casilla esta libre para una caja: valida, no hay cajas, no está el otro jugador (una caja si que se puede mover a donde hay agua, porque cae encima del agua y la tapa)


#### Ejercicio 3

Completa las siguientes funciones

In [ ]:
def is_valid(coord):
    """ Checks if a cell is valid.
    Uses the same variables defined in exercise 1
    
    Parameters
    ----------
    coord -- coordinate of the cell to check
    
    Returns
    --------
    Returns True if the cell is  is within the limits of the board and there is a 0 in this position
    """    
    return False

def is_box_free(coord):
    """ Checks if a cell is valid and it is possible to put a box in that coord
    Uses the same variables defined in exercise 1
    
    Parameters
    ----------
    coord -- coordinate of the cell to check
    
    Returns
    --------
    Returns True if the cell is valid and there is not a box or any player
    """  
    return False

def is_player_free(coord):
    """ Checks if a cell is valid and it is possible to put the current player in that coord
    Uses the same variables defined in exercise 1
    
    Parameters
    ----------
    coord -- coordinate of the cell to check
    
    Returns
    --------
    Returns True if the cell is valid and there is not a box, water or the other player
    """   
    return False





In [ ]:
# prueba aquí las funciones






## Funciones de actualización

Las funciones de actualización son la de mover y la de cambiar turno.
En la función de mover, se recibe un movimiento y si puede o es necesario empuja las cajas y se desplaza.

Para hacer la función de mover es posible (pero no obligatorio) apoyarse en dos funciones básicas de actualización.



- move_player: 
    - Recibe un movimiento ([1,0] abajo, [-1,0] arriba, [0,1] derecha, [0,-1] izquierda)
    - Suma el movimiento a la posición del jugador actual.
    - Si la casilla resultante es libre:
        - Se el jugador actual para a valer la casilla resultante
        - Si el jugador actual llega al destino:
            El jugador actual valdrá None y cambiará el turno
     - Devuelve una lista de jugadores actualizada (una nueva lista), sino devuelve la lista de jugadores sin modificar.

- move_boxes: 
    - Recibe el movimiento ([1,0] abajo, [-1,0] arriba, [0,1] derecha, [0,-1] izquierda)
    - Suma el movimiento a la posición del jugador actual.
    - Si la posición resultante coincide con una caja del set:
        - Aplica ese movimiento también a la caja.
        - Si la nueva posición de la caja está libre:
            - Saca del set la caja a empujar, actualiza sus coordenadas.
            - Si en la nueva posición hay agua elimina esa coordenada del set de agua.
            - Si no vuelve a meter la caja actualizada en el set de cajas
    - Devuelve los sets de cajas y de agua actualizados.
    
- change_turn:
    - Devuelve turno complementado (True -> False, False -> True)
    
    
Estas funciones no actualizan los valores del estado, solo devuelven los valores nuevos. Si el movimiento no se puede efectuar devuelve una copia de las variables sin actualizar.


#### Ejercicio 4

Completa las siguientes funciones:

In [ ]:
def move(mov):
    """ Mueve un jugador

    Move the player and boxes if the movement is valid

    Parameters
    ---------
    mov -- direction of the movement
    
    Returns
    --------
    Returns a new list with the values of the players, a new set of boxes and a new set of water
    """  
    
    return None


def move_player(mov):
    """ Mueve un jugador

    Move the player if the movement is valid

    Parameters
    ---------
    mov -- direction of the movement
    
    Returns
    --------
    Returns a new list with the values of the players

    """  
    
    return None

def move_boxes(mov):
    """ Mueve cajas

    Move the boxes if the movement is valid


    Parameters
    ---------
    mov -- direction of the movement
    
    Returns
    --------
    Returns a new set of boxes and water

    """  
    
    return None

def change_turn():
    return False if turn else True







In [ ]:
# Resetea los valores iniciales para hacer pruebas
turn = True 
players = [[3,4],[2,1]]
boxes = set([(2,2)])
water = set([(2,4)])


print(players, boxes,water)
# Se intenta mover hacia arriba, no pasa nada
players = move_player([-1,0])
print(turn, players, boxes,water)

turn = change_turn()
# jugador 2 empuja la caja y se mueve
boxes,water = move_boxes([0,1])
players = move_player([0,1])
print(turn, players, boxes,water)

# jugador 2 empuja la caja y se mueve, desaparece el agua y la caja
boxes,water = move_boxes([0,1])
players = move_player([0,1])
print(turn, players, boxes,water)

# jugador 2 se mueve
players = move_player([0,1])
print(turn, players, boxes,water)

# jugador 2 se mueve hacia arriba, entra a la meta
players = move_player([-1,0])
print(turn, players, boxes,water)


# jugador 1 se mueve hacia arriba
turn = change_turn()
players = move_player([-1,0])
print(turn, players, boxes,water)

# jugador 1 se mueve hacia arriba, entra a la meta, los dos jugadores deben de ser None
players = move_player([-1,0])
print(turn, players, boxes,water)


'''
Debe imprimir
[[3, 4], [2, 1]] {(2, 2)} {(2, 4)}
True [[3, 4], [2, 1]] {(2, 2)} {(2, 4)}
False [[3, 4], [2, 2]] {(2, 3)} {(2, 4)}
False [[3, 4], [2, 3]] set() set()
False [[3, 4], [2, 4]] set() set()
False [[3, 4], None] set() set()
True [[2, 4], None] set() set()
True [None, None] set() set()

'''
" "


## Funciones de representación gráfica

Va a haber una función pinta juego. **Está función se proporciona por el profesor** pero el alumno puede modificarla, si quiere, para hacer que el juego sea más atractivo. Usando conceptos más avanzados de HTML como canvas o lo que se le ocurra. También puede usar otro tipo de representaciones gráficas no basadas en HTML pero no se garatiza que funcionen el resto de códigos proporcionados por el profesor.

Simplemente se crea una tabla HTML.

- `<td></td>` define una celda
- `<tr></tr>` define una fila
- `<table></table>` define una tabla
- `<img ></img>` permite insertar una imagen.
- `<style> </style>` permite definir un estilo, cambiar la apariencia visual de un elemento HTML


Con

```HTML
<style> img.game {width: 50px !important; height: 50px !important;}</style>
<img class="game" src="./sprites/hombre.jpg" alt="" ></img></td>
```

Se está creando el estilo game de elemento img, para que todos los elementos que usen este estilo sean de 50px x 50px. Si posteriormente se inserta una imagen y se especifíca que la clase (estilo) es game, automáticamente será del tamaño fijado anteriormente.

--------------------------


Para hacer el código más modular, más fácilmente adaptable a otros juegos o imágenes, la representación de cada elemento se va a codificar en forma de diccionario, de esta forma solo habría que modificar el diccionario para cambiar como se dibuja.

Esto valdría para nuevos juegos en los que hubiese un número distinto de elementos dibujables.



In [ ]:
element_image = {
    "grass": "./sprites/grass.png",
    "wall": "./sprites/stones.png",
    "destination": "./sprites/meta.jpg",
    "player1": "./sprites/gazpacho.jpg",
    "player2": "./sprites/pincho.jpg",
    "box": "./sprites/caja.jpg",
    "water": "./sprites/water.jpg"
}

element_image["grass"]

In [ ]:
from IPython.display import display
from ipywidgets import HTML


# Resetea los valores iniciales para hacer pruebas
turn = True 
players = [[3,4],[2,1]]
boxes = set([(2,2)])
water = set([(2,4)])


def get_html():
    """ Shows a graphic representation of the game.

    Returns a String that contains HTML

    """ 
    
    
    height = len(board)
    width = len(board[0])
    
    html_string = "<style> img.game {width: 50px !important; height: 50px !important;}</style><table>"
    


    new_row = "<tr>"
    end_row = "</tr>"
    
    for i in range(height):
        html_string+=new_row
        for j in range(width):
            
            content = get_content((i,j))
            if content[1] is None:
                drawing = element_image[content[0]]
            else:
                drawing = element_image[content[1]]
            html = '<td><img class="game" src=%s alt=""></img></td>' % drawing     
            
            
            html_string+=html
        html_string+=end_row
            
    html_string += "</table>"
        
                
    return html_string

HTML(get_html())

## Prueba gráfica del funcionamiento de las funciones

In [ ]:
from IPython.display import display

# Resetea los valores iniciales para hacer pruebas
turn = True 
players = [[3,4],[2,1]]
boxes = set([(2,2)])
water = set([(2,4)])

display(HTML(get_html()))


turn = change_turn()
# jugador 2 empuja la caja y se mueve
boxes,water = move_boxes([0,1])
players = move_player([0,1])
display(HTML(get_html()))

# jugador 2 empuja la caja y se mueve, desaparece el agua y la caja
boxes,water = move_boxes([0,1])
players = move_player([0,1])
display(HTML(get_html()))


'''
# jugador 2 se mueve
players = move_player([0,1])
display(HTML(get_html()))

# jugador 2 se mueve hacia arriba, entra a la meta
players = move_player([-1,0])
display(HTML(get_html()))
'''
# jugador 1 se mueve
turn = change_turn()
players = move_player([-1,0])
display(HTML(get_html()))

# jugador 1 se mueve hacia arriba, entra a la meta
players = move_player([-1,0])
display(HTML(get_html()))

# Funciones de carga de niveles (Opcional)

Va a haber una función que carga niveles a partir de ficheros de texto.
El profersor proporciona una versión, pero el alumno podría mejorarla.

Se va a usar este formato:


| Nombre elemento del juego     | Caracter usado para la codificación | 
|---	        |---	|
|Wall   	    | #   	|
|Player1   	|@   	|
|Player2   	|$   	|
|Box   	    |+   	|
|Water   	    |-   	|
|Destination   	    |*   	|
|Grass   	    |(espacio)   	|

-----------
A la hora de copiar un nivel, ojo a los espacios de más al final de cada fila.

Se puede partir una cadena en filas usando **split** y usando como parámetro el separador de filas "\n".
Después en con un par de bucles **for** podemos recorrer los caracteres de cada una de las filas del nivel.
- por cada fila del nivel se va a añadir una fila al tablero
    - si en un caracter es '#' se va a añadir un 1 al tablero, sino se va a añadir 0
    - si un caracter es '@' se va a crear el jugador1 con esas coordenadas.
    - si un caracter es '$' se va a crear el jugador2 con esas coordenadas
    - si un caracter es '+' se va a crear una caja con esas coordenadas.
    - si un caracter es '-' se va a crear agua con esas coordenadas.
    - si un caracter es '\*' se va a crear la meta con esas coordenadas.
    
Haz la función de manera incremental, primero intenta devolver el tablero, despues el tablero y el jugador, despues el tablero, el jugador y las cajas etc

------

Al igual que con el diccionario que asocia un elemento del juego con un gráfico, se puede hacer un diccionario que asocie un caracter con un elemento del juego, de esta forma la codificación de los niveles puede adaptarse y cambiar sin tener que modificar el código en exceso. 

In [ ]:
file_element = {
    "#": ['wall', None],
    "@": ['grass', 'player1'],
    "$": ['grass', 'player2'],
    "+": ['grass', 'box'],
    "*": ['destination', None],
    "-": ['grass', 'water'],
    " ": ['grass', None]
}



In [ ]:
nivel1=("######\n"
        "#  #*#\n"
        "#$+ -#\n"
        "# # @#\n"
        "######")

print(nivel1)


In [ ]:
# Ejercicio opcional, trata de hacer tu versión

def load_level(str_level):
    """ analiza un string con un nivel construye tablero, jugador, cajas y destinos y los devuelve

    Devuelve tablero, jugador, cajas, destinos
    
    Parámetros:
    str_level -- representación en modo texto del mapa

    """ 
    return None,None,None,None





'''
Versión del profesor
'''
def load_level(str_level):
    level = str_level.split("\n")
    
    board = []
    boxes = set()
    water = set()
    destination = () 
    
    player1,player2 = None, None
    
    i =0
    for row in level:
        board_row = []
        j =0
        for caracter in row:
            element = file_element[caracter]
            
            if element[0] == "wall":
                board_row.append(1)
            else:
                board_row.append(0)
                
            if element[1] =="player1":
                player1 = [i,j]
            
            if element[1] =="player2":
                player2 = [i,j]
            
            if element[1] =="box":
                boxes.add((i,j))
                
            if element[1] =="water":
                water.add((i,j))
                
            if element[0] == "destination":
                destination = (i,j)
                
            j+=1
        i+=1
        
        board.append(tuple(board_row))
    
    return tuple(board),destination,[player1,player2],boxes,water,True
    


In [ ]:
board,destination,players,boxes,water,turn= load_level(nivel1) 
HTML(get_html())


In [ ]:
board,players

In [ ]:
boxes,water


# Pruebas adicionales

El código de estas funciones será la base del código de la práctica obligatoria por lo que habrá que testearlo correctamente.

A continuación se presentan unos casos de prueba básicos que pueden ser ampliados por el alumno con más casos y más niveles.

### unittest

El framework **unittest** se desarrolló inspirado en JUnit (el framework de testing de Java) y es similar a los framework de testing de otros lenguajes. Soporta la automatización de pruebas y la posibilidad de poder compartir mecanísmos de *setup* (inicialización de variables, configuraciones etc) y *shutdown* (cerrado de ficheros etc). También se pueden agregar test formando colecciones.


Los conceptos más importantes de **unittest** son los siguientes:
- *test fixture*. Las preparaciones necesarias para realizar una o más pruebas. Y cualquier operación posterior de limpieza. Algunos ejemplos son creación de bases de datos, directorios, inicialización de variables, arrancar un servidor.

- *test case*. Es caso de prueba. Comprueba el funcionamiento en un caso particular de entrada. **unittest** proporciona una clase base llamada *TestCase* que puede ser usada para crear nuevos casos de prueba.

- *test suite*. Es una colección de *test case*, de otros *test suites* o de ambas cosas a la vez. Es una colección que debe ser ejecutada a la vez.

- *test runner*. Es un componente que ejecuta los test y proporciona los resultados al usuario. El *test runner* puede tener interfaz gráfica o de tipo texto.



Para crear un caso de prueba hay que heredar de *unittest.TestCase*

Dentro de un caso de prueba hay tantos métodos como se quiera, cuando empiecen por la cadena "test" serán pruebas unitarias.

Dentro de cada prueba unitaria tiene que haber un *assert* que son métodos que comprueban que se obtiene un determinado resultado. Hay de varios tipos:
- assertEqual()
- assertTrue()
- etc

El método *setUp()* define instrucciones que serán ejecutadas **antes** de cada prueba unitaria.
El método *tearDown()* define instrucciones que serán ejecutadas **después** de cada prueba unitaria.


Para ejecutar unas pruebas podemos crear un *test suite* al que le añadimos tantos casos de prueba como queramos y por último *TextTestRunner* es un objeto con un método *run* que puede ejecutar una *test suite*.

In [ ]:
nivel_empuja=  ("########\n"
                "#  +   #\n"
                "# +@++ #\n"
                "#  +   #\n"
                "#  -   #\n"
                "########")
import unittest


class TestEmpuja(unittest.TestCase):
    
    def setUp(self):
        global board,destination,players,boxes,water,turn        
        board,destination,players,boxes,water,turn= load_level(nivel_empuja) 


    def test_arriba(self):
        """
        No se puede mover porque hay pared arriba
        """
        new_boxes,new_water = move_boxes([-1,0])
        self.assertEqual(new_boxes, boxes)
        
    def test_abajo(self):
        """
        Se empuja la caja al agua.
        El agua desaparece
        La caja en (3,3) desaparece
        """
        new_boxes,new_water = move_boxes([1,0])
        self.assertEqual(len(new_water), 0)
        self.assertEqual(new_boxes, {(1, 3), (2, 2), (2, 5), (2, 4)})
        


def suite():
    suite = unittest.TestSuite()
    suite.addTest(TestEmpuja('test_arriba'))
    suite.addTest(TestEmpuja('test_abajo'))

    
    return suite






if __name__ == '__main__':
    #unittest.main(argv=['first-arg-is-ignored'], exit=False)
    
    runner = unittest.TextTestRunner()
    runner.run(suite())

#### Ejercicio final

Define algunos tableros con distintos casos de prueba para asegurarte que tus funciones son correctas.